In [149]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, make_scorer
from sklearn.tree import DecisionTreeClassifier as dtc 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [217]:
log_directory_tls12 = "/home/admin/C2_TLS/datasets/ms/TLS1.2/short_cert/zeek_logs"
log_files12 = [f for f in os.listdir(log_directory_tls12) if f.endswith('.log')]

log_directory_tls13 = "/home/admin/C2_TLS/datasets/ms/TLS1.3/zeek_logs"
log_files13 = [f for f in os.listdir(log_directory_tls13) if f.endswith('.log')]

df_list = []

for log_file in log_files12:
    file_path = os.path.join(log_directory_tls12, log_file)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    df = df.dropna()
    df_list.append(df)

for log_file in log_files13:
    file_path = os.path.join(log_directory_tls13, log_file)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    df = df.dropna()
    df_list.append(df)



headers = ['client_hello_len','client_hello_ext_num','server_hello_len','server_hello_ext_num','tls_version']
df_ms = pd.concat(df_list, ignore_index=True)
df_ms.shape
df_ms = shuffle(df_ms, random_state=42)
df_ms.reset_index(drop=True, inplace=True)
# df_ms = df_ms.iloc[:40000]
df_ms = df_ms.set_axis(headers, axis=1)
df_ms['c2'] = 1
df_ms['source'] = 'MS'
df_ms_noise = df_ms.iloc[10001:11000]
df_ms_noise.loc[:, 'c2'] = 0

In [218]:
log_directory_tls12 = "/home/admin/C2_TLS/datasets/tranco/TLS1.2/zeek_logs"
log_files12 = [f for f in os.listdir(log_directory_tls12) if f.endswith('.log')]

log_directory_tls13 = "/home/admin/C2_TLS/datasets/tranco/TLS1.3/zeek_logs"
log_files13 = [f for f in os.listdir(log_directory_tls13) if f.endswith('.log')]

# # List all .log files in the directory
# log_files = [f for f in os.listdir(log_directory) if f.endswith('.log')]

# List to hold the DataFrames
df_list = []

for log_file in log_files12:
    file_path = os.path.join(log_directory_tls12, log_file)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    df = df.dropna()
    df_list.append(df)

for log_file in log_files13:
    file_path = os.path.join(log_directory_tls13, log_file)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    df = df.dropna()
    df_list.append(df)

#Load Tranco dataset (i.e., Benign traffic)
df_tranco= pd.concat(df_list, ignore_index=True)
df_tranco = shuffle(df_tranco, random_state=42)
df_tranco.reset_index(drop=True, inplace=True)
# df_tranco = df_tranco.iloc[:40000]
df_tranco = df_tranco.set_axis(headers, axis=1)
df_tranco['c2'] = 0
df_tranco['source'] = 'tranco'
# df_tranco_noise = df_tranco.iloc[10001:11000]
# df_tranco_noise.loc[:, 'c2'] = 1

In [227]:
log_directory_benign = "/home/admin/C2_TLS/datasets/DoHBrw/benign/zeek_logs"

# List all .log files in the directory
log_files_benign = [f for f in os.listdir(log_directory_benign) if f.endswith('.log')]

# List to hold the DataFrames
df_list = []

# Loop through each log file and load it into a DataFrame
for log_file in log_files_benign:
    file_path = os.path.join(log_directory_benign, log_file)
    
    # Read the log file into a DataFrame (assuming CSV or tab-delimited format)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)  # or change delimiter depending on the log format
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    # Remove rows where any column contains NaN (or previously "-")
    df = df.dropna()
    # Append the DataFrame to the list
    df_list.append(df)

#Load Tranco dataset (i.e., Benign traffic)
df_doh_benign= pd.concat(df_list, ignore_index=True)
df_doh_benign = shuffle(df_doh_benign, random_state=42)
df_doh_benign.reset_index(drop=True, inplace=True)
# df_doh_benign = df_doh_benign.iloc[:20000]
df_doh_benign = df_doh_benign.set_axis(headers, axis=1)
df_doh_benign['c2'] = 0
df_doh_benign['source'] = 'DoH'
# df_doh_benign_noise = df_doh_benign.iloc[10001:11000]
# df_doh_benign_noise.loc[:, 'c2'] = 1
###########################################################################################################################
log_directory_malicious = "/home/admin/C2_TLS/datasets/DoHBrw/malicious/zeek_logs"

# List all .log files in the directory
log_files_malicious = [f for f in os.listdir(log_directory_malicious) if f.endswith('.log')]

# List to hold the DataFrames
df_list = []

# Loop through each log file and load it into a DataFrame
for log_file in log_files_malicious:
    file_path = os.path.join(log_directory_malicious, log_file)
    
    # Read the log file into a DataFrame (assuming CSV or tab-delimited format)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)  # or change delimiter depending on the log format
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    # Remove rows where any column contains NaN (or previously "-")
    df = df.dropna()
    # Append the DataFrame to the list
    df_list.append(df)

#Load Tranco dataset (i.e., Benign traffic)
df_doh_malicious= pd.concat(df_list, ignore_index=True)
df_doh_malicious = shuffle(df_doh_malicious, random_state=42)
df_doh_malicious.reset_index(drop=True, inplace=True)
# df_doh_malicious = df_doh_malicious.iloc[:20000]
df_doh_malicious = df_doh_malicious.set_axis(headers, axis=1)
df_doh_malicious['c2'] = 1
df_doh_malicious['source'] = 'DoH'
df_doh_malicious_noise = df_doh_malicious.iloc[10001:11000]
df_doh_malicious_noise.loc[:, 'c2'] = 0

df_doh = pd.concat([df_doh_malicious,df_doh_benign])

In [228]:
df_all = pd.concat([df_tranco,df_ms, df_doh], ignore_index=True)
df_all = shuffle(df_all, random_state=42)
df_all.reset_index(drop=True, inplace=True)

In [229]:
df_all

,client_hello_len,client_hello_ext_num,server_hello_len,server_hello_ext_num,tls_version,c2,source
0,589,14.0,118.0,2.0,3.0,0,tranco
1,508,12.0,94.0,3.0,3.0,0,DoH
2,252,10.0,118.0,2.0,3.0,1,DoH
3,508,10.0,118.0,2.0,3.0,1,MS
4,179,6.0,61.0,4.0,3.0,1,MS
...,...,...,...,...,...,...,...
1391932,580,14.0,118.0,2.0,3.0,0,tranco
1391933,587,14.0,118.0,2.0,3.0,0,tranco
1391934,182,10.0,76.0,7.0,2.0,0,tranco
1391935,508,10.0,118.0,2.0,3.0,1,MS


# Split Data

In [230]:
X = df_all.loc[:,headers].values.astype(np.float64)
y = df_all.loc[:,"c2"].values
y

array([0, 0, 1, ..., 0, 1, 0])

In [231]:
X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=42,  
                                   test_size=0.25,  
                                   shuffle=True) 

# Training

In [232]:
rf_model = RandomForestClassifier(max_depth=20, n_estimators = 5, max_leaf_nodes=500, n_jobs=4, random_state=42, bootstrap=False)                              
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test data: {accuracy}")
print(classification_report(y_test,y_pred))
importances = rf_model.feature_importances_
feature_importances = pd.DataFrame({'feature': headers, 'importance': importances})
feature_importances = feature_importances.sort_values('importance', ascending=False)
feature_importances

Accuracy on test data: 0.9985487880224723
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    219862
           1       1.00      1.00      1.00    128123

    accuracy                           1.00    347985
   macro avg       1.00      1.00      1.00    347985
weighted avg       1.00      1.00      1.00    347985



,feature,importance
1,client_hello_ext_num,0.599213
3,server_hello_ext_num,0.172671
0,client_hello_len,0.135480
2,server_hello_len,0.054375
4,tls_version,0.038261


In [215]:
# import pickle
# pickle.dump(rf_model, open("RF_new.pkl", 'wb'))

# Load Distribution

In [171]:
def get_logs(log_directory):
    log_files = [f for f in os.listdir(log_directory) if f.endswith('.log')]
    # List to hold the DataFrames
    df_list = []

    # Loop through each log file and load it into a DataFrame
    for log_file in log_files:
        file_path = os.path.join(log_directory, log_file)
        
        # Read the log file into a DataFrame (assuming CSV or tab-delimited format)
        df = pd.read_csv(file_path, sep="\t", header=None, skiprows=8, low_memory=False)  # or change delimiter depending on the log format
        df = df.iloc[:-1]
        df.replace("-", pd.NA, inplace=True)
        # Remove rows where any column contains NaN (or previously "-")
        df = df.dropna()
        # Append the DataFrame to the list
        df_list.append(df)
    df_list= pd.concat(df_list, ignore_index=True)
    sum = df_list.iloc[:,0].astype("float64").sum(axis=0)
    mean = df_list.iloc[:,3].mean(axis=0)
    return sum

In [172]:
DoHBrw_benign = "/home/admin/C2_TLS/datasets/DoHBrw/benign/zeek_logs_stats"
DoHBrw_malicious = "/home/admin/C2_TLS/datasets/DoHBrw/malicious/zeek_logs_stats"

ms_tls2_short_zeek_logs = "/home/admin/C2_TLS/datasets/ms/TLS1.2/short_cert/zeek_logs_stats"
ms_tls2_zeek_logs = "/home/admin/C2_TLS/datasets/ms/TLS1.2/long_cert/zeek_logs_stats"
ms_tls3_zeek_logs = "/home/admin/C2_TLS/datasets/ms/TLS1.3/zeek_logs_stats"

tranco_tls12_zeek_logs = "/home/admin/C2_TLS/datasets/tranco/TLS1.2/zeek_logs_stats"
tranco_tls13_zeek_logs = "/home/admin/C2_TLS/datasets/tranco/TLS1.3/zeek_logs_stats"

mean_DoHBrw_benign = get_logs(DoHBrw_benign)
mean_DoHBrw_malicious = get_logs(DoHBrw_malicious)

mean_ms_tls2_short_zeek_logs = get_logs(ms_tls2_short_zeek_logs)
mean_ms_tls2_zeek_logs = get_logs(ms_tls2_zeek_logs)
mean_ms_tls3_zeek_logs = get_logs(ms_tls3_zeek_logs)

mean_tranco_tls12_zeek_logs = get_logs(tranco_tls12_zeek_logs)
mean_tranco_tls13_zeek_logs = get_logs(tranco_tls13_zeek_logs)


print(mean_DoHBrw_benign, mean_DoHBrw_malicious, mean_ms_tls2_short_zeek_logs, mean_ms_tls2_zeek_logs, mean_ms_tls3_zeek_logs,mean_tranco_tls12_zeek_logs,
      mean_tranco_tls13_zeek_logs )


1.6199748275862071 0.8047527499999999 10.040702666666666 10.047599416666667 9.023684583333335 2.1072224285714283 2.1474964285714284
